### 데이터 로드 및 preprocessing

In [1]:
import pandas as pd

In [13]:
df = pd.read_parquet('데이터/02~23/news_oil_02~23.parquet')

In [5]:
# df.drop('url', axis=1, inplace = True)

In [6]:
# df['date'][233] = '2003. 2. 19.'

In [14]:
df['date'] = pd.to_datetime(df['date'])
df

,title,date
0,Building Beefy Blocks - Hardcore Tech! 360 A-B...,2002-01-01
1,PM: Guyana Oil Production Frustrated by Neig...,2002-01-09
2,A Reclusive Oil Baron Builds 'Taj Mahal' in Sa...,2002-01-10
3,SCOTLAND | BP fined £1m for safety offences,2002-01-18
4,Gas prices expected to spike,2002-01-27
...,...,...
5939,Brazil environment minister demands 'ceiling' ...,2023-12-25
5940,Oil drops almost 2% as investors watch Red Sea...,2023-12-27
5941,"Oil prices shed 10% in 2023 as supply, demand ...",2023-12-29
5942,Why Oil Prices Never Surged to $100 This Year,2023-12-29


In [27]:
# df = df.sort_values(by='date')


In [36]:
# df = df.reset_index(drop = True)
# df.to_parquet('데이터/02~23/news_oil_02~23.parquet', compression = 'gzip', engine = 'pyarrow', index = False)

In [32]:
pd.read_csv("SP_vol.csv")

,date,return
0,1986-01-31,1.138226
1,1986-02-07,1.959957
2,1986-02-14,2.346657
3,1986-02-21,2.325644
4,1986-02-28,2.393233
...,...,...
1990,2024-04-19,-0.802495
1991,2024-04-26,-0.867596
1992,2024-05-03,-0.826431
1993,2024-05-10,-0.793951


In [18]:
import pandas as pd
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, trainers
from transformers import PreTrainedTokenizerFast
from tqdm import tqdm

LOWERCASE = False
VOCAB_SIZE = 10000

# Creating Byte-Pair Encoding tokenizer
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))

# Adding normalization and pre_tokenizer
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

# Adding special tokens and creating trainer instance
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)

def train_corp_iter():
    """
    A generator function for iterating over the DataFrame in chunks.
    """
    for i in range(0, len(df), 1000):
        yield df.iloc[i : i + 1000]['title'].tolist()

# Training from iterator
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)

tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

# Tokenize the entire DataFrame
tokenized_texts = []

for title in tqdm(df['title'].tolist()):
    tokenized_texts.append(tokenizer.tokenize(title))


100%|████████████████████████████████████████████████████████████████████████████| 5944/5944 [00:00<00:00, 7175.93it/s]


In [ ]:
# Convert tokens to token IDs
token_ids = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tokenized_texts]

# Untokenize the text by decoding the token IDs
untokenized_texts = [tokenizer.decode(ids) for ids in token_ids]

# Print original, tokenized, and untokenized texts
for original, tokenized, untokenized in zip(df['title'].tolist(), tokenized_texts, untokenized_texts):
    print(f"Original: {original}")
    print(f"Tokenized: {tokenized}")
    print(f"Untokenized: {untokenized}")
    print()